# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Expressions and statements”

The following table compares [statements](https://en.wikipedia.org/wiki/Statement_(computer_science)) and [expressions](https://en.wikipedia.org/wiki/Expression_(computer_science)):

| | Returns something? | Has side-effects? |
|- |- |-
| Statements | Never | Always |
| Expressions| Always | Rarely |

> When working in languages such as C# and VB.NET, we often don’t think about the differences between statements and expressions, as these language mix and match both features. I consider both of these languages to be primarily statement-based languages, in that statements are easy to acheive, but expressions aren’t.


## Difficulties with statements

Here is a `void` of C# statements:

In [ ]:
public void DescribeAge(int age)
{
    string ageDescription = null;
    var greeting = "Hello";

    if(age < 18) ageDescription = "Child!";
    else if (age < 65) greeting = "Adult";

    Console.WriteLine($"{greeting}! You are a “{ageDescription}.”");
}

The intent of the above is confusing:

- is `greeting` supposed to be assigned what appears to be value for `ageDescription`?
- is there a reason why ages 65 and above are ignored?
- there is no defense against the default `null` value of `ageDescription` being passed to the string in `Console.WriteLine`.

>Your initial instinct might be to say that no one makes mistakes like that, and that this is a strawman example. …statements are _weak_. Compilers have no understanding that there’s any relationship between all the branches of the if/else block.

The compiler cannot know that the intent here is to use these branches to assign a value to the _same_ variable. We can better express this intent in C# by:

1. separating the _query_ portion of the above from the _command_ portion
2. eliminating the use of variables by branching to `return` statements


### running `DescribeAge(65)`

Running `DescribeAge(65)` is, um, exceptional:


In [ ]:
DescribeAge(65)

Hello! You are a “.”


The default `null` value of `ageDescription` appears as `string.Empty`.


In [ ]:
private static string GetText(int age)
{
    if(age < 18) return "Child";
    else if(age < 65) return "Adult";
    else return "OAP!";
}

public void DescribeAgeV2(int age)
{
    var ageDescription = GetText(age);
    var greeting = "Hello";

    Console.WriteLine($"{greeting}! You are a “{ageDescription}.”");
}

Now `GetText` is the _query_ portion with the single responsibility of querying for a value. `DescribeAgeV2` is the _command_ portion, setting `ageDescription` without confusion by calling the query and ultimately commanding `Console.WriteLine`.

The use of `return` statements in the `if`-`then` branches forces _all_ `age` values to be covered.

What may be more familiar in the world of F# is the use of [_local functions_](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/local-functions) in C#:


In [ ]:
public void DescribeAgeV3(int age)
{
    string GetText(int age)
    {
        if(age < 18) return "Child";
        else if(age < 65) return "Adult";
        else return "OAP!";
    }

    var ageDescription = GetText(age);
    var greeting = "Hello";

    Console.WriteLine($"{greeting}! You are a “{ageDescription}.”");
}

This relatively new C# syntax can be confusing what with all the angle brackets (similar to nested-`if` optical confusion).

Let’s try `age = 65` again with these new methods:


In [ ]:
DescribeAgeV2(65)

Hello! You are a “OAP!.”


In [ ]:
DescribeAgeV3(65)

Hello! You are a “OAP!.”


## Using expressions in F#&nbsp;

We can translate quite directly `DescribeAgeV3` to F#:


In [ ]:
#!fsharp

open System

let describeAge age =
    let ageDescription =
        if age < 18 then "Child!"
        elif age < 65 then "Adult!"
        else "OAP!"

    let greeting = "Hello"
    Console.WriteLine $"{greeting}! You are a “{ageDescription}.”"

In [ ]:
#!fsharp

describeAge 65

Hello! You are a “OAP!.”


## Introducing `unit`

Remember that `describeAge` is an F# function; it is not a C# method of `void`. This means that the last line of `describeAge` returns something:


In [ ]:
#!fsharp

(describeAge 65) = ()

Hello! You are a “OAP!.”


True

The `True` thrown out above is saying that the return value of `describeAge` is equal to `()` and:

>The `unit` type has a single value, and that value is indicated by the token `()`.
>
> [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/unit-type)]

In .NET we have `Task` and `Task<T>` because of the use of `void` (or `Sub`) in the statement-based languages. Because of `unit` in F#, this expression-based language only needs `Task<T>` (`Task<unit>`).


## Discarding results

With `unit` F# ensures that you always have a return type.

>F# also tells you that you might be doing something wrong if you call a function and don’t use the return value.

For example:


In [ ]:
#!fsharp

let getMyValue() =
    39

let getMyValues() =
    getMyValue()
    getMyValue()
    getMyValue()

getMyValues()

39

The first two calls to `getMyValue` should be underlined with warning colors with the message:

```console
 The result of this expression has type 'int' and is implicitly ignored.
 Consider using 'ignore' to discard this value explicitly,
 e.g. 'expr |> ignore',
 or 'let' to bind the result to a name,
 e.g. 'let result = expr'.
```


To intentionally _discard_ these results we use the `ignore` [inline](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/functions/inline-functions) function ([GitHub](https://github.com/fsharp/fsharp/blob/d6244d4a1b7bc4efce42bc050ad08384cdce0320/src/fsharp/FSharp.Core/prim-types.fs#L426)):


In [ ]:
#!fsharp

let getMyValuesV2() =
    ignore(getMyValue())
    ignore(getMyValue())
    getMyValue()

getMyValues()

39

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.235701+3881a96164de75fca84f5f11027f3606b7878044Build date: 2021-07-11T04:06:39.6100964Zhttps://github.com/dotnet/interactive
